In [6]:
import gc
import re
import nltk
import regex
import numpy as np
import pandas as pd
import nltk.corpus
import json
from nltk import SnowballStemmer

In [7]:
ItemInfo_attrsJSON = pd.read_hdf("ItemInfo_test_attrsJSON.h")
ItemInfo_attrsJSON['attrsJSON'].fillna("", inplace=True)

In [8]:
print(ItemInfo_attrsJSON.shape)

# Create dicitonary with common attributes
attributes_dictionary = {}

for i, j in enumerate(ItemInfo_attrsJSON['attrsJSON']):
    if i % 100000 == 0 :
        print('processed docs ',i)
#     print(i , json)
#     print("=======")
    j = str(j)
    if len(j) > 0:
        for key in json.loads(j).keys():
#             key = jsn.lower()
#             key = regex.sub(r"\p{P}", "", key)
            if key not in attributes_dictionary:
                attributes_dictionary[key] = 1
            else:
                attributes_dictionary[key] += 1



(1315205, 2)
processed docs  0
processed docs  100000
processed docs  200000
processed docs  300000
processed docs  400000
processed docs  500000
processed docs  600000
processed docs  700000
processed docs  800000
processed docs  900000
processed docs  1000000
processed docs  1100000
processed docs  1200000
processed docs  1300000


In [9]:
def clearJson(jsn):
    jsn = jsn.lower() # to lower case
    jsn = regex.sub(r"[^\P{P}-]+", " ", jsn) # replace punctuiation by space
    jsn = regex.sub(r"\s+", " ", jsn) # replace multiple spaces by one
    return jsn

def deleteAttrFromJson(jsn):
    pattern_replace_pair_list = [
        (r"(?<=\W|^)%s(?=\W|$)"%a, "") for a in attributes_dictionary.keys()
    ]
    
    for pattern, replace in pattern_replace_pair_list:
        jsn = regex.sub(pattern, replace, jsn)
    return jsn

def getOnlyAttrDescription(jsn):
    jsn = deleteAttrFromJson(jsn)
    jsn = clearJson(jsn)
    return jsn

def getProcessedJson(jsn):
    jsn = clearJson(jsn)
    return jsn

def getOnlyAttrTitles(jsn):
    attrTitles = ""
    for attr in attributes_dictionary.keys():
        if attr in jsn:
            attrTitles += attr + " "
    return attrTitles.lower()

processed_attrsJSON = []
processed_attrsJSON_titles = []
processed_attrsJSON_description = []

for i, j in enumerate(ItemInfo_attrsJSON['attrsJSON']):
    if i % 100000 == 0 :
        print('processed docs ',i)
    jsn = str(j)
    processed_attrsJSON.append(getProcessedJson(jsn).strip())
    processed_attrsJSON_titles.append(getOnlyAttrTitles(jsn).strip())
    processed_attrsJSON_description.append(getOnlyAttrDescription(jsn).strip())

ItemInfo_attrsJSON.drop('attrsJSON', axis=1, inplace=True)
ItemInfo_attrsJSON['attrsJSON'] = processed_attrsJSON
ItemInfo_attrsJSON['attrsJSON_titles'] = processed_attrsJSON_titles
ItemInfo_attrsJSON['attrsJSON_description'] = processed_attrsJSON_description
print('storing')
ItemInfo_attrsJSON.to_hdf("ItemInfo_test_attrsJSON_processed.h", 'w')
print('done')

processed docs  0
processed docs  100000
processed docs  200000
processed docs  300000
processed docs  400000
processed docs  500000
processed docs  600000
processed docs  700000
processed docs  800000
processed docs  900000
processed docs  1000000
processed docs  1100000
processed docs  1200000
processed docs  1300000
storing
done


In [5]:
del ItemInfo_attrsJSON
gc.collect()

30